In [1]:
from rw_ve_plot import *
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
plt.style.use(r"./RW_visualization.mplstyle")


In [2]:
# import warnings filter
# from pandas.core.common import SettingWithCopyWarning
import warnings
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.options.mode.chained_assignment = None 

In [ ]:
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Drop rows if efficacy does not exist
vaccine_detail = vaccine_detail[vaccine_detail['efficacy_in_%'] != 'X']

# Remove all vaccine
vaccine_detail = vaccine_detail[vaccine_detail.vaccine!='All vaccine']

In [ ]:
plot_vaccine_efficacy(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_variants_group(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_ave_group(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_ave_group_for_variants(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_variants_and_ave_group(vaccine_detail, save_figure=False)

Average vaccine efficacy

In [ ]:
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Drop rows if efficacy does not exist
vaccine_detail = vaccine_detail[vaccine_detail['efficacy_in_%'] != 'X']

plot_average_vaccine_efficacy(vaccine_detail, save_figure=False)

Use my reproduced vaccine efficacy

In [10]:
# Run vaccine_efficacy_CIs.ipynb to update the data
# df1 = pd.read_excel(r'C:\Users\dipes\python\jupyter_notebook_files\Vaccine details.xlsx')
vaccine_detail = pd.read_excel('rw_vaccine_details.xlsx')
# vaccine_detail = pd.read_excel('./Vaccine/RW_vaccine_details.xlsx')
vaccine_detail.columns = vaccine_detail.columns.str.strip().str.lower().str.replace(' ', '_').\
    str.replace('(', '').str.replace(')', '')

# Remove all vaccine
vaccine_detail = vaccine_detail[vaccine_detail.vaccine!='All vaccine']

In [11]:
# Replace the vaccine efficacy and Ci by my estimations
vaccine_detail['efficacy_in_%'] = vaccine_detail['rw_efficacy_in_%']
vaccine_detail['lower'] = vaccine_detail['rw_lower']
vaccine_detail['upper'] = vaccine_detail['rw_upper']

In [12]:
# Drop empty vaccine efficacy
vaccine_detail = vaccine_detail[~np.isnan(vaccine_detail['efficacy_in_%'])]

In [ ]:
vaccine_detail

In [ ]:
plot_vaccine_efficacy_ave_group_for_original(vaccine_detail, save_figure=False)

In [ ]:
plot_vaccine_efficacy_ave_group_for_variants(vaccine_detail, save_figure=False)

In [52]:
def plot_vaccine_efficacy_ave_platform(vaccine_detail, save_fig=False):
    """
    Create separate forest plots for vaccine efficacy for each platform
    
    Parameters:
    vaccine_detail (pd.DataFrame): DataFrame containing vaccine details and efficacy data
    save_fig (bool): Whether to save the figure
    """
    # Get unique platforms
    platform_unique = np.sort(vaccine_detail['platform'].unique())
    
    # Plot for each platform
    for platform_idx, platform in enumerate(platform_unique):
        platform_data = vaccine_detail[vaccine_detail['platform'] == platform]
        vaccine_name_unique = np.sort(platform_data['vaccine'].unique())
        
        # Create color palette for this platform's plot
        palette = sns.color_palette('Set1', n_colors=len(vaccine_name_unique)+2)
        palette = palette[0:5] + palette[6:14] + palette[15::]
        
        # Initialize index tracking
        index = np.array([0])
        index_temp = 0
        
        # Create figure for this platform
        fig = plt.figure(figsize=(10, len(platform_data.vaccine)*0.35))
        ax = fig.add_subplot(111)
        
        # For each vaccine in this platform
        print(f'Platform: {platform}')
        for i in vaccine_name_unique:
            vaccine_data = platform_data[platform_data['vaccine'] == i]
            
            # Get efficacy and confidence bounds
            try:
                efficacy = vaccine_data['vaccine_efficacy']
            except:
                efficacy = vaccine_data['efficacy_in_%']
                
            try:
                lower_bounds = vaccine_data['lower_bound']
            except:
                lower_bounds = vaccine_data['lower']
                
            try:
                upper_bounds = vaccine_data['upper_bound']
            except:
                upper_bounds = vaccine_data['upper']
            
            # Handle missing values in bounds
            lower_bounds[lower_bounds.index[
                (lower_bounds == 'X') | (lower_bounds == '-')].tolist()] \
                = efficacy[lower_bounds.index[
                    (lower_bounds == 'X') | (lower_bounds == '-')].tolist()]
            upper_bounds[upper_bounds.index[
                (upper_bounds == 'X') | (upper_bounds == '-')].tolist()] \
                = efficacy[upper_bounds.index[
                    (upper_bounds == 'X') | (upper_bounds == '-')].tolist()]
            
            # Convert to numpy arrays
            efficacy = efficacy.to_numpy()
            lower_bounds = lower_bounds.to_numpy()
            upper_bounds = upper_bounds.to_numpy()
            
            # Plot points
            for j in range(len(efficacy)):
                if lower_bounds[j] == 0 and upper_bounds[j] == 0:
                    ax.errorbar(efficacy[j], j+index[-1],
                               xerr=np.array([[0, 0]]).T, 
                               fmt='o', 
                               color=palette[len(index)-1])
                else:
                    ax.errorbar(efficacy[j], j+index[-1],
                               xerr=np.array([[max(0, efficacy[j]-lower_bounds[j]),
                                             max(0, upper_bounds[j]-efficacy[j])]]).T, 
                               fmt='o', 
                               color=palette[len(index)-1])
                index_temp += 1
            index = np.append(index, index_temp)
        
        # Add reference lines
        ax.plot([30, 30], [-1, index_temp], 'k--')
        ax.plot([50, 50], [-1, index_temp], 'k--')
        
        # Customize plot
        # ax.set_title(f'Platform: {platform}', fontsize=16)
        ax.set_xlabel('Efficacy (%)', fontsize=14)
        ax.set_xticks(np.arange(0, 110, 10))
        ax.set_yticks(index[0:-1])
        ax.set_yticklabels(vaccine_name_unique, fontsize=12)
        ax.set_ylim([-1, index_temp])
        ax.set_xlim([0, 100])
        
        # Invert y-axis and add grid
        plt.gca().invert_yaxis()
        plt.gca().yaxis.grid()
        
        # Add tight layout
        plt.tight_layout()
        
        # Save figure if requested
        if save_fig:
            plt.savefig(f'RW2025_vaccine_efficacy_{platform.replace("/", "_").lower()}.pdf', bbox_inches='tight')
        
        plt.show()

In [ ]:
plot_vaccine_efficacy_ave_platform(vaccine_detail, save_fig=True)

## Calculate the overall VE categorized by platform

In [ ]:
vaccine_detail

In [77]:
def VE_platform(vaccine_detail, platform):
    vaccine_data = vaccine_detail[vaccine_detail['platform'] == platform].copy()
    print(vaccine_data)

    xv = vaccine_data['no_of_participants_in_vaccine_group']
    xp = vaccine_data['no_of_participants_in_placebo_groupcontrol_group']
    xve = vaccine_data['no_of_infected_in_vaccine_group']
    xpe = vaccine_data['no_of_infected_in_placebo_group']

    xv_all = np.sum(xv)
    xp_all = np.sum(xp)
    xve_all = np.sum(xve)
    xpe_all = np.sum(xpe)

    r1 = xve_all/xv_all
    r2 = xpe_all/xp_all
    theta = r1/r2
    VE = 1-theta
    
    variance = 1/xve_all - 1/xv_all + 1/xpe_all - 1/xp_all
    se = np.sqrt(variance)
    lb = 1-theta*np.exp(1.96*se)
    hb = 1-theta*np.exp(-1.96*se)

    print(f'{platform}: {VE*100:.1f}% [{lb*100:.1f}%, {hb*100:.1f}%]')


In [ ]:
VE_platform(vaccine_detail, 'Inactivated')

In [ ]:
for platform in sorted(vaccine_detail['platform'].unique()):
    VE_platform(vaccine_detail, platform)